In [1]:
import theano
import numpy as np
import time
import skimage.io

import sys
sys.path.insert(0, '/home/flipvanrijn/Workspace/Dedicon-Thesis/')

%load_ext autoreload
%autoreload 2

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: Quadro K2200


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(precision=5, suppress=True)

# Settings
EMBEDDING_SIZE = 512

In [4]:
%%time
from networks.keras.vocab import Vocab

vocab = Vocab('/home/flipvanrijn/Workspace/Dedicon-Thesis/downloads/coco_train_dataset.json')
print 'Vocabulary loaded'

Processing /home/flipvanrijn/Workspace/Dedicon-Thesis/downloads/coco_train_dataset.json...
Reduced words from 25128 to 8853 with threshold 5
Vocabulary loaded
CPU times: user 26.9 s, sys: 1.3 s, total: 28.2 s
Wall time: 28.4 s


In [ ]:
%%time
from keras.models import Sequential
from keras.layers.core import Dense, Activation, TimeDistributedDense
from keras.layers.recurrent import LSTM, Bidirectional
from keras.regularizers import l2

model = Sequential()
model.add(Bidirectional(LSTM, vocab.token_count, 512, return_sequences=True))
model.add(TimeDistributedDense(512, vocab.token_count))
model.compile(loss='binary_crossentropy', optimizer='adagrad', class_mode='binary')

In [ ]:
from networks.keras.batchgenerator import BatchGenerator
from IPython import embed

bg = BatchGenerator(vocab)
for Xs, Ys in bg.get_data(batch_size=50):
    
    model.fit(Xs, Xs, batch_size=50, nb_epoch=1)
    
    preds = model.predict(Xs)
    
    

In [6]:
%%time
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Activation, TimeDistributedDense, Dropout
from keras.layers.recurrent import LSTM

model = Sequential()
model.add(Embedding(vocab.token_count, 256))
model.add(LSTM(256, 128, activation='sigmoid', inner_activation='hard_sigmoid', return_sequences=True))
model.add(Dropout(0.5))
model.add(TimeDistributedDense(128, vocab.token_count))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adagrad')

CPU times: user 40.3 s, sys: 471 ms, total: 40.7 s
Wall time: 40.6 s


In [7]:
from networks.keras.batchgenerator import BatchGenerator
from IPython import embed

bg = BatchGenerator(vocab, batch_size=2500)
for Ys, Xs in bg.get_data():
    print 'Batch {}/{}'.format(bg.current_batch + 1, bg.total_batches)
    model.fit(Xs, Ys, nb_epoch=1, show_accuracy=True)

Batch 1/166
Epoch 0
2500/2500 [==============================] - 59s - loss: 0.0076 - acc: 0.0046    
Batch 2/166
Epoch 0
2500/2500 [==============================] - 57s - loss: 0.0020 - acc: 0.0167    
Batch 3/166
Epoch 0
2500/2500 [==============================] - 56s - loss: 0.0012 - acc: 0.0272    
Batch 4/166
Epoch 0
2500/2500 [==============================] - 56s - loss: 0.0009 - acc: 0.0318    
Batch 5/166
Epoch 0
2500/2500 [==============================] - 55s - loss: 0.0008 - acc: 0.0285    
Batch 6/166
Epoch 0
2500/2500 [==============================] - 55s - loss: 0.0008 - acc: 0.0296    
Batch 7/166
Epoch 0
2500/2500 [==============================] - 55s - loss: 0.0007 - acc: 0.0248    
Batch 8/166
Epoch 0
2500/2500 [==============================] - 55s - loss: 0.0007 - acc: 0.0328    
Batch 9/166
Epoch 0
2500/2500 [==============================] - 55s - loss: 0.0007 - acc: 0.0312    
Batch 10/166
Epoch 0
2500/2500 [==============================] - 55s - loss: 0.00

KeyboardInterrupt: 

In [3]:
import glob

files = glob.glob('/media/Data/flipvanrijn/datasets/text/1billion/*')

In [4]:
from networks.keras.vocab import Vocab
vocab = Vocab(files)

In [5]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dropout, Dense, Activation
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(Embedding(vocab.word_count, 512))
model.add(LSTM(512, 512, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512, 512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(512, vocab.word_count))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

MemoryError: ('Error allocating 996378624 bytes of device memory (out of memory).', "you might consider using 'theano.shared(..., borrow=True)'")

from utils.batch import Batch

maxlen = 10
b = Batch(vocab, ['/media/Data/flipvanrijn/datasets/text/1billion/news.en-00032-of-00100'], 100)

In [ ]:
from IPython import embed
for X, y in b.get_data(maxlen=maxlen):
    Xs = np.zeros((len(X), maxlen, vocab.word_count), dtype=np.bool)

Python 2.7.10 (default, Jun 25 2015, 09:27:35) 
Type "copyright", "credits" or "license" for more information.

IPython 3.2.0 -- An enhanced Interactive Python.
?         -> Introduction and overview of IPython's features.
%quickref -> Quick reference.
help      -> Python's own help system.
object?   -> Details about 'object', use 'object??' for extra details.

In [1]: Xs.shape
Out[1]: (0, 10, 2187839)

In [2]: quit()

Python 2.7.10 (default, Jun 25 2015, 09:27:35) 
Type "copyright", "credits" or "license" for more information.

IPython 3.2.0 -- An enhanced Interactive Python.
?         -> Introduction and overview of IPython's features.
%quickref -> Quick reference.
help      -> Python's own help system.
object?   -> Details about 'object', use 'object??' for extra details.

KeyboardInterrupt

KeyboardInterrupt

KeyboardInterrupt
